In [20]:
from PIL import Image, ImageFilter
import glob
import os
import shutil

In [21]:
new_path = 'StreetView/train/noBlur'
blured_path = 'StreetView/train/Blur'
label_path ='StreetView/train/label'
folder= 'part1' 

In [24]:
def bluring_resize_images(new_path,blured_path,label_path,folder):
    temp = 'temp'
    label = 'label'
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    if not os.path.exists(blured_path):
        os.makedirs(blured_path)
    if not os.path.exists(label_path):
        os.makedirs(label_path)
    if not os.path.exists(temp):
        os.makedirs(temp)
    # loop over existing images and resize
    # change path to your path
    
    images_list=os.listdir(folder)
    i=0
    # the only solution to keep input/output with same orde is to make sure they
    # have the same names
    for item in images_list:
        image = Image.open(folder+"/"+item)   
        image.save(temp +'/' + str(i)+'.jpg', 'JPEG', quality=90) 
        i+=1
        
    for filename in glob.glob(temp+'/' +'*.jpg'): #path of raw images
        img = Image.open(filename).resize((224,224))
        # save resized images to new folder with existing filename
        img.save('{}{}{}'.format(new_path,'/',os.path.split(filename)[1]))
        
        #****applying GaussianBlur filter
        image = Image.open(filename)
        width, height = image.size
        img_dic={'0':{'left':10,'top':int( height / 4),'right':400,'bottom':int(3 * height / 4)},
         '1':{'left':400,'top':int( height / 4),'right':10,'bottom':int(3 * height / 4)}}
        smol_image1= image.crop((img_dic['0']['left'],img_dic['0']['top'],img_dic['0']['right'],img_dic['0']['bottom']))
        blurred_image1 = smol_image1.filter(ImageFilter.GaussianBlur(10))
        image.paste(blurred_image1,(img_dic['0']['left'], img_dic['0']['top']))
        image = image.resize((224,224))
        image.save('{}{}{}'.format(blured_path,'/',os.path.split(filename)[1]))
        
        #*******generating label**********
        label_img = np.zeros((height, width, 3), dtype = "uint8")
        label_img = Image.fromarray(label_img)
        x,y = smol_image1.size
        whiteFrame = 255 * np.ones((y,x,3), np.uint8)
        whiteFrame = Image.fromarray(whiteFrame)
        label_img.paste(whiteFrame,(img_dic['0']['left'], img_dic['0']['top']))
        label_img = label_img.resize((224,224))
        label_img.save('{}{}{}'.format(label_path,'/',os.path.split(filename)[1]))
    
    shutil.rmtree(temp)

In [25]:
bluring_resize_images(new_path,blured_path,label_path,folder)